![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTKsyOlLfV6FsVPWyjaGl4jyjaKD20EoxiWIkbIswXOJRIruaun&s)
<BR>Prof: Julián Gómez Cuadrado

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import pyspark
import matplotlib.pyplot as plt
import pandas as pd

    

In [0]:
%sh
wget "https://www.gutenberg.org/files/2000/2000-0.txt" -O /tmp/quijote.txt

In [0]:

dbutils.fs.mv("file:/tmp/quijote.txt", "dbfs:/data/quijote.txt")

In [0]:
file = 'dbfs:/data/quijote.txt'
fichero = sc.textFile(file)

In [0]:
def limpiarStr(x):
  punc='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ '
  lowercased_str = x.lower()
  for ch in punc:
    lowercased_str = lowercased_str.replace(ch, '')
  return lowercased_str

# Hola esto es un título

In [0]:
# Esto es un  comentario 
lista = fichero.flatMap(lambda linea: linea.split(" ")) \
              .map (limpiarStr) \
             .map(lambda palabra: (palabra,1)) \
             .reduceByKey(lambda x, y: x + y)

In [0]:
lista

# Pasamos de RDD a DataFrame

In [0]:
df=lista.toDF(['palabra','frecuencia'])
display(df)

In [0]:
print(type(df))

In [0]:
#sqlContext.registerDataFrameAsTable(df, "tablaQuijote")
df.createOrReplaceTempView("tablaQuijote")

In [0]:
print (df.count())

# Jugamos con la API de SparkSQL

In [0]:
%sql
SELECT * FROM tablaQuijote ORDER BY frecuencia DESC LIMIT 20

In [0]:
sqlDF = spark.sql("SELECT * FROM tablaQuijote ORDER BY frecuencia DESC LIMIT 20")

#sqlDF = spark.sql("select * from tablaQuijote order by frecuencia desc limit 100")

#sqlDF = spark.sql("select * from tablaQuijote where palabra IN ('quijote','sancho','dulcinea','rocinante','rucio') ORDER BY frecuencia DESC")

#sqlDF = spark.sql("select * from tablaQuijote where palabra like '%quijote%' ORDER BY veces DESC")

display(sqlDF)

# Representación gráfica con Matplotlib

In [0]:
%matplotlib inline
pdDF=sqlDF.toPandas()
pdDF.plot(figsize = (20,10), kind = "bar", color = "red",x = "palabra", y = "frecuencia", legend = False)
plt.xlabel("", fontsize = 18)
plt.ylabel("Personaje", fontsize = 18)
plt.title("nº de apariciones", fontsize = 28)
plt.xticks(size = 18)
plt.yticks(size = 18)
plt.show()

##SQL

In [0]:
%sql
select * from tablaQuijote where palabra IN ('quijote','sancho','dulcinea','rocinante','rucio') ORDER BY frecuencia DESC